In [193]:
pwd

'/home/ec2-user/MY - Text Class'

In [194]:
# !pip install s3fs

In [195]:
# bucket = 'w210awsbucket/fridge'
# data_key = 'df_fridge.csv'
# data_location = 's3://{}/{}'.format(bucket,data_key)

# pd.read_csv(data_location)

In [196]:
# !pip install -q transformers einops accelerate langchain bitsandbytes
# !pip install sentencepiece xformers trl transformers

In [197]:
# !pip install spacy

In [198]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModel, AutoModelForSequenceClassification, BertForSequenceClassification, AdamW, BertConfig, BertTokenizer, BertTokenizerFast
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from datasets import load_dataset
from trl import SFTTrainer
import spacy
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

In [199]:
df = pd.read_csv('joined_labels.csv')

# label_mapping = {
#     'Delivery/Packaging': 0,
#     'Quality': 1,
#     'Design/Functionality': 2,
#     'Information': 3,
#     'Other': 4,
#     'Service': 5,
#     'Value': 6
# }


In [200]:
df.head()

,reviewText,labels,product_category,num_labels
0,Used it 3 times...now leaking oil...what a POS.,Quality,mower,1
1,I don't like how the throttle works. It is har...,Design/Functionality,mower,2
2,Here's the real deal on these new cheap Poulan...,Design/Functionality,mower,2
3,If you are looking for a manual push mower you...,Design/Functionality,mower,2
4,DO NOT BUY\nI used this twice and it stopped w...,Quality,mower,1


In [201]:
nb_labels = 3
model_id = 'bert-base-uncased'
#model_id = 'bert-large-uncased'

In [202]:
df = df[df['num_labels']<= (nb_labels-1)]

In [203]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.index.values, 
                                                  df.num_labels.values, 
                                                  test_size=0.10, 
                                                  random_state=42, 
                                                  stratify=df.num_labels.values)

X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train, 
                                                  test_size=0.111, 
                                                  random_state=42, 
                                                  stratify=y_train)

In [204]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.loc[X_test, 'data_type'] = 'test'

In [205]:
tokenizer = BertTokenizer.from_pretrained(model_id, 
                                          do_lower_case=True)

In [206]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].reviewText.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].reviewText.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].reviewText.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].num_labels.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].num_labels.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].num_labels.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ec2-user/anaconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [207]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [208]:
len(dataset_train), len(dataset_val), len(dataset_test)

(1867, 234, 234)

In [209]:
model = BertForSequenceClassification.from_pretrained(model_id,
                                                      #num_labels=len(label_dict),
                                                      num_labels=nb_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [210]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [211]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/home/ec2-user/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [212]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [213]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [214]:
from sklearn.metrics import precision_recall_fscore_support

def f1_score_byClass(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels_flat, preds_flat, average=None)
    
    return precision, recall, f1_score
    #return f1_score

In [215]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [216]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu') #use cpu to diagnose error
model.to(device)

print(device)

cpu


In [217]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [218]:
from tqdm.notebook import tqdm

In [219]:
# for epoch in tqdm(range(1, epochs+1)):
    
#     model.train()
    
#     loss_train_total = 0

#     progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
#     for batch in progress_bar:

#         model.zero_grad()
        
#         batch = tuple(b.to(device) for b in batch)
        
#         inputs = {'input_ids':      batch[0],
#                   'attention_mask': batch[1],
#                   'labels':         batch[2],
#                  }       

#         outputs = model(**inputs)
        
#         loss = outputs[0]
#         loss_train_total += loss.item()
#         loss.backward()

#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

#         optimizer.step()
#         scheduler.step()
        
#         progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
#     #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
#     torch.save(model.state_dict(), f'data_volume2/finetuned_BERT_Large_uncased_DQD_epoch_{epoch}.model')
        
#     tqdm.write(f'\nEpoch {epoch}')
    
#     loss_train_avg = loss_train_total/len(dataloader_train)            
#     tqdm.write(f'Training loss: {loss_train_avg}')
    
#     val_loss, predictions, true_vals = evaluate(dataloader_validation)
#     val_f1 = f1_score_func(predictions, true_vals)
#     tqdm.write(f'Validation loss: {val_loss}')
#     tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [220]:
model = BertForSequenceClassification.from_pretrained(model_id,
                                                      num_labels=nb_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [221]:
#model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))
model.load_state_dict(torch.load('data_volume2/finetuned_BERT_base_uncased_DQD_epoch_2.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [186]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [222]:
_, predictions_test, true_test = evaluate(dataloader_test)

In [98]:
df.head()

,reviewText,labels,product_category,num_labels,data_type
0,Used it 3 times...now leaking oil...what a POS.,Quality,mower,1,not_set
1,I don't like how the throttle works. It is har...,Design/Functionality,mower,2,train
2,Here's the real deal on these new cheap Poulan...,Design/Functionality,mower,2,train
3,If you are looking for a manual push mower you...,Design/Functionality,mower,2,val
4,DO NOT BUY\nI used this twice and it stopped w...,Quality,mower,1,val


In [99]:
# n_class = nb_labels  # number of classes
# label_list = []
# for i in range(n_class):
#     label_list.append(df.columns[i+4]) #add 4 to start at first manual label column which is Delivery/Packaging

In [100]:
# label_dict = {}
# for k in range(len(label_list)):
#     label_dict[label_list[k]] = k #do not need to subtract 1 here to match above labeling system since range already subtracts 1
# label_dict

In [101]:
label_dict = {
    'Delivery/Packaging': 0,
    'Quality': 1,
    'Design/Functionality': 2
    # ,'Information': 3,
    # 'Other': 4
    # ,'Service': 5,
    # 'Value': 6
}

In [192]:
accuracy_per_class(predictions, true_vals)

Class: Delivery/Packaging
Accuracy: 67/73

Class: Quality
Accuracy: 74/85

Class: Design/Functionality
Accuracy: 65/76



In [223]:
accuracy_per_class(predictions_test, true_test)

Class: Delivery/Packaging
Accuracy: 66/73

Class: Quality
Accuracy: 67/85

Class: Design/Functionality
Accuracy: 63/76



In [187]:
f1_score_func(predictions, true_vals)

0.880478920621355

In [224]:
f1_score_func(predictions_test, true_test)

0.837337071580769

In [188]:
precision, recall, f1_score = f1_score_byClass(predictions, true_vals)

In [225]:
precision, recall, f1_score = f1_score_byClass(predictions_test, true_test)

In [226]:
categories = ['Delivery/ Packaging', 'Quality', 'Design/Functionality']
for i, k in enumerate(precision):
    print(f'{categories[i]} Precision: {k:.3f}')

Delivery/ Packaging Precision: 0.892
Quality Precision: 0.807
Design/Functionality Precision: 0.818


In [227]:
categories = ['Delivery/ Packaging', 'Quality', 'Design/Functionality']
for i, k in enumerate(recall):
    print(f'{categories[i]} Recall: {k:.3f}')

Delivery/ Packaging Recall: 0.904
Quality Recall: 0.788
Design/Functionality Recall: 0.829


In [228]:
categories = ['Delivery/ Packaging', 'Quality', 'Design/Functionality']
for i, k in enumerate(f1_score):
    print(f'{categories[i]} F1-score: {k:.3f}')

Delivery/ Packaging F1-score: 0.898
Quality F1-score: 0.798
Design/Functionality F1-score: 0.824


In [40]:
ex = df.reviewText[1]
ex

"For what boils down to a mini-fridge the kegerator is priced way too high. The one I received had a crooked door. All of the metal is rusted after two years, don't believe any stainless claims."

In [41]:
ex2 = "I purchased this for my husband as a present. We are happy with the unit except for the fact that it arrived dented in three places"
ex2

'I purchased this for my husband as a present. We are happy with the unit except for the fact that it arrived dented in three places'

In [42]:
def bert_predict(text, model, tokenizer, n_class):
    
    inputs = tokenizer.encode_plus(
        text, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        pad_to_max_length=True, 
        max_length=256, 
        return_tensors='pt'
    )
    model.eval()
    with torch.no_grad():    
        output = model(**inputs)
    
    return np.argmax(output[0].detach().numpy(), axis= 1).item()



In [43]:
def predict(row):
    review_text = row['reviewText']
    predictions = bert_predict(review_text, model, tokenizer, n_class)
    return predictions

In [44]:
from tqdm import tqdm

In [45]:
len(df)

1329

In [46]:
sample = df

In [47]:

tqdm.pandas()
# Apply the predict function to each row of the DataFrame
#sample['predictions'] = sample.apply(predict, axis=1)
sample['predictions'] = sample.progress_apply(predict, axis=1)

  0%|                                                  | 0/1329 [00:00<?, ?it/s]/home/ec2-user/anaconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|███████████████████████████████████████| 1329/1329 [01:17<00:00, 17.14it/s]


In [48]:
sample

,overall,reviewText,reviewID,GPT Result,Delivery/Packaging,Quality,Design/Functionality,Other,Value,Categories,Label,data_type,predictions
0,3,I purchased this for my husband as a present. ...,1365,Delivery/Packaging,True,False,False,False,False,1,0,train,0
1,2,For what boils down to a mini-fridge the keger...,1374,Quality,False,True,False,False,False,1,1,val,0
3,3,The product's casters immediately broke - just...,474503,Quality,False,True,False,False,False,1,1,train,2
4,1,sent me a cheap one. not the stainless steel o...,474504,Quality,False,True,False,False,False,1,1,train,1
5,1,I too have had a hard time getting this produc...,7337,Packaging,True,False,False,False,False,1,0,val,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,3,unit arrived with a few dents. markers were dr...,598187,Delivery/Packaging,True,False,False,False,False,1,0,train,0
1373,1,Came shipped with huge dent on side. Runs cons...,599287,Loading...,True,False,False,False,False,1,0,train,0
1374,3,"The freezer itself work very good. However, th...",599583,Loading...,True,False,False,False,False,1,0,val,0
1375,2,Item came with dents in right lower end damaged.,599584,Loading...,True,False,False,False,False,1,0,train,0


In [49]:
#add predictions for entire df_fridge dataset for about 6000 entries
tableau_data = pd.read_csv('df_fridge.csv')
tableau_data['predictions'] = tableau_data.progress_apply(predict, axis=1)

  0%|                                                  | 0/6250 [00:00<?, ?it/s]/home/ec2-user/anaconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|███████████████████████████████████████| 6250/6250 [06:03<00:00, 17.20it/s]


In [50]:
tableau_data.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,...,rank,also_view,details,main_cat,similar_item,date,price,imageURL,imageURLHighRes,predictions
0,3,NaN,False,"09 7, 2009",A38TZ2NJ97APSO,B00006JKZX,NaN,J. Lussier,I purchased this for my husband as a present. ...,Happy with product,...,"['>#25,110 in Appliances (See top 100)', '>#23...","['B0170NWHDM', 'B000HJVYDQ', 'B0170O0D82', 'B0...",{},Appliances,"class=""a-bordered a-horizontal-stripes a-spa...","June 6, 2007",NaN,[],[],0
1,2,NaN,False,"10 23, 2007",A35QRTOA6YLJXF,B00006JKZX,NaN,Cliff H,For what boils down to a mini-fridge the keger...,"Lousy Fit & Finish, High Price",...,"['>#25,110 in Appliances (See top 100)', '>#23...","['B0170NWHDM', 'B000HJVYDQ', 'B0170O0D82', 'B0...",{},Appliances,"class=""a-bordered a-horizontal-stripes a-spa...","June 6, 2007",NaN,[],[],0
2,1,NaN,False,"12 13, 2012",ACFO1BTLWH43O,B00006JKZX,NaN,Jeff,You can get this exact same product at walmart...,Same thing better price,...,"['>#25,110 in Appliances (See top 100)', '>#23...","['B0170NWHDM', 'B000HJVYDQ', 'B0170O0D82', 'B0...",{},Appliances,"class=""a-bordered a-horizontal-stripes a-spa...","June 6, 2007",NaN,[],[],1
3,3,NaN,True,"04 1, 2013",A2MAD6PK9AUCS1,B0000AYKNR,NaN,Matthew Overin,The product's casters immediately broke - just...,Not the Best Value,...,"['>#17,384 in Appliances (See top 100)', '>#16...","['B00YW50UX6', 'B0170NWHDM', 'B0170O0D82', 'B0...",{},Appliances,"class=""a-bordered a-horizontal-stripes a-spa...","October 18, 2007",NaN,[],[],2
4,1,NaN,False,"02 9, 2012",A7CCVYNSG2Q5B,B0000AYKNR,NaN,annoyed,sent me a cheap one. not the stainless steel ...,wrong one,...,"['>#17,384 in Appliances (See top 100)', '>#16...","['B00YW50UX6', 'B0170NWHDM', 'B0170O0D82', 'B0...",{},Appliances,"class=""a-bordered a-horizontal-stripes a-spa...","October 18, 2007",NaN,[],[],1


In [51]:
type(tableau_data['predictions'][1])

numpy.int64

In [52]:
labels_dict = {0: 'Delivery/ Packaging', 1: 'Quality', 2: 'Design/Functionality'}

In [53]:
tableau_predCat = tableau_data.copy()

In [54]:
tableau_predCat['predictions'] = tableau_predCat['predictions'].map(labels_dict)

In [58]:
import ast
tableau_predCat['imageURLHighRes'] = tableau_predCat['imageURLHighRes'].apply(ast.literal_eval)

In [59]:
tableau_predCat['firstImageURL'] = tableau_predCat['imageURLHighRes'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

In [60]:
tableau_predCat.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,...,also_view,details,main_cat,similar_item,date,price,imageURL,imageURLHighRes,predictions,firstImageURL
0,3,NaN,False,"09 7, 2009",A38TZ2NJ97APSO,B00006JKZX,NaN,J. Lussier,I purchased this for my husband as a present. ...,Happy with product,...,"['B0170NWHDM', 'B000HJVYDQ', 'B0170O0D82', 'B0...",{},Appliances,"class=""a-bordered a-horizontal-stripes a-spa...","June 6, 2007",NaN,[],[],Delivery/ Packaging,None
1,2,NaN,False,"10 23, 2007",A35QRTOA6YLJXF,B00006JKZX,NaN,Cliff H,For what boils down to a mini-fridge the keger...,"Lousy Fit & Finish, High Price",...,"['B0170NWHDM', 'B000HJVYDQ', 'B0170O0D82', 'B0...",{},Appliances,"class=""a-bordered a-horizontal-stripes a-spa...","June 6, 2007",NaN,[],[],Delivery/ Packaging,None
2,1,NaN,False,"12 13, 2012",ACFO1BTLWH43O,B00006JKZX,NaN,Jeff,You can get this exact same product at walmart...,Same thing better price,...,"['B0170NWHDM', 'B000HJVYDQ', 'B0170O0D82', 'B0...",{},Appliances,"class=""a-bordered a-horizontal-stripes a-spa...","June 6, 2007",NaN,[],[],Quality,None
3,3,NaN,True,"04 1, 2013",A2MAD6PK9AUCS1,B0000AYKNR,NaN,Matthew Overin,The product's casters immediately broke - just...,Not the Best Value,...,"['B00YW50UX6', 'B0170NWHDM', 'B0170O0D82', 'B0...",{},Appliances,"class=""a-bordered a-horizontal-stripes a-spa...","October 18, 2007",NaN,[],[],Design/Functionality,None
4,1,NaN,False,"02 9, 2012",A7CCVYNSG2Q5B,B0000AYKNR,NaN,annoyed,sent me a cheap one. not the stainless steel ...,wrong one,...,"['B00YW50UX6', 'B0170NWHDM', 'B0170O0D82', 'B0...",{},Appliances,"class=""a-bordered a-horizontal-stripes a-spa...","October 18, 2007",NaN,[],[],Quality,None


In [60]:
#tableau_data.to_csv('tableau_fridge_data.csv')

In [61]:
tableau_predCat.to_excel('tableau_fridge_data_v2.xlsx')

In [108]:
tableau_manual_label = pd.read_csv('Fridge_Label.csv')

In [110]:
tableau_manual_label.head()

,overall,reviewText,reviewID,GPT Result,Delivery/Packaging,Quality,Design/Functionality,Other,Value,Categories,Label
0,3,I purchased this for my husband as a present. ...,1365,Delivery/Packaging,True,False,False,False,False,1,1
1,2,For what boils down to a mini-fridge the keger...,1374,Quality,False,True,False,False,False,1,2
2,1,You can get this exact same product at walmart...,474186,Other,False,False,False,True,True,1,4
3,3,The product's casters immediately broke - just...,474503,Quality,False,True,False,False,False,1,2
4,1,sent me a cheap one. not the stainless steel o...,474504,Quality,False,True,False,False,False,1,2


In [ ]:
labels_dict2 = {0: 'Delivery/ Packaging', 1: 'Quality', 2: 'Design/Functionality'}